In [2]:
import pandas as pd
import os
from pathlib import Path

In [2]:
# Define column mapping to match data.md convention
column_mapping = {
    'pm10': 'pm_sepuluh',
    'so2': 'sulfur_dioksida',
    'co': 'karbon_monoksida',
    'o3': 'ozon',
    'no2': 'nitrogen_dioksida',
    'critical': 'parameter_pencemar_kritis',
    'categori': 'kategori',
    'lokasi_spku': 'stasiun'
}

# Define column order to match data.md
column_order = [
    'periode_data', 'tanggal', 'stasiun', 'pm_sepuluh',
    'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida',
    'max', 'parameter_pencemar_kritis', 'kategori'
]


final_year = 2019

# Define the folder path and years to combine
ispu_folder = Path("data/ISPU")
years_to_combine = [i for i in range(2015, final_year + 1, 1)]

# List to store dataframes
dfs = []

# Load and standardize files that contain the desired years
for file_path in ispu_folder.glob("*.csv"):
    # Check if any of the desired years is in the filename
    for year in years_to_combine:
        if str(year) in file_path.name:
            df_temp = pd.read_csv(file_path)
            
            print(f"Loaded: {file_path.name}")
            print(f"  Original columns: {list(df_temp.columns)}")
            
            # Rename columns to match convention
            df_temp = df_temp.rename(columns=column_mapping)
            
            # Add missing columns with None
            for col in column_order:
                if col not in df_temp.columns:
                    df_temp[col] = None
            
            # Reorder columns to match data.md
            df_temp = df_temp[column_order]
            
            print(f"  Standardized to: {list(df_temp.columns)}")
            print(f"  Rows: {len(df_temp)}\n")
            
            dfs.append(df_temp)
            break

print(f"Total files loaded: {len(dfs)}")

# Combine all dataframes
df_combined = pd.concat(dfs, ignore_index=True)

Loaded: indeks-standar-pencemaran-udara-(ispu)-tahun-2018-komponen-data.csv
  Original columns: ['periode_data', 'tanggal', 'pm10', 'so2', 'co', 'o3', 'no2', 'max', 'critical', 'categori', 'lokasi_spku']
  Standardized to: ['periode_data', 'tanggal', 'stasiun', 'pm_sepuluh', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'parameter_pencemar_kritis', 'kategori']
  Rows: 365

Loaded: indeks-standar-pencemaran-udara-(ispu)-tahun-2019-komponen-data.csv
  Original columns: ['periode_data', 'tanggal', 'pm10', 'so2', 'co', 'o3', 'no2', 'max', 'critical', 'categori', 'lokasi_spku']
  Standardized to: ['periode_data', 'tanggal', 'stasiun', 'pm_sepuluh', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'parameter_pencemar_kritis', 'kategori']
  Rows: 345

Loaded: indeks-standar-pencemaran-udara-(ispu)-tahun-2016-komponen-data.csv
  Original columns: ['periode_data', 'tanggal', 'stasiun', 'pm10', 'so2', 'co', 'o3', 'no2', 'max', 'critical', 'c

In [3]:
# Replace empty strings and "---" with NULL (NaN)
df_combined = df_combined.replace(['', '---', ' ', '  '], pd.NA)
df_combined = df_combined[df_combined['kategori'] != 'TIDAK ADA DATA']

df_combined['tanggal'] = pd.to_datetime(
    df_combined['tanggal'], 
    format='mixed',  # Allows multiple formats
    dayfirst=False,  # Month comes first in ambiguous cases
    errors='raise'
)

# Convert pollutant columns to integers (pd.NA will remain as NaN for nullable Int64)
numeric_columns = ['pm_sepuluh', 'sulfur_dioksida', 
                   'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max']

for col in numeric_columns:
    df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce').astype('Int64')

In [4]:
import re

# Function to extract station ID (DKIx format)
def extract_station_id(station_name):
    if pd.isna(station_name):
        return station_name
    # Extract DKI followed by a number
    match = re.search(r'DKI\d+', str(station_name), re.IGNORECASE)
    if match:
        return match.group(0).upper()
    return station_name

# Apply normalization to stasiun column
df_combined['stasiun'] = df_combined['stasiun'].apply(extract_station_id)

In [5]:
df_combined['ID'] = df_combined['tanggal'].astype(str) + '_' + df_combined['stasiun'].astype(str)
df_combined = df_combined[['ID'] + column_order]
df_combined = df_combined.sort_values(by=['tanggal']).reset_index(drop=True)
df_combined = df_combined.ffill()

In [6]:
df_combined.info()

<class 'pandas.DataFrame'>
RangeIndex: 4663 entries, 0 to 4662
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   ID                         4663 non-null   str           
 1   periode_data               4663 non-null   int64         
 2   tanggal                    4663 non-null   datetime64[us]
 3   stasiun                    4663 non-null   str           
 4   pm_sepuluh                 4663 non-null   Int64         
 5   sulfur_dioksida            4663 non-null   Int64         
 6   karbon_monoksida           4663 non-null   Int64         
 7   ozon                       4663 non-null   Int64         
 8   nitrogen_dioksida          4663 non-null   Int64         
 9   max                        4663 non-null   Int64         
 10  parameter_pencemar_kritis  4663 non-null   str           
 11  kategori                   4663 non-null   str           
dtypes: Int64(6), date

In [7]:
df_combined.to_csv('data/ispu_2015-2019.csv', index=False)

# QC

In [8]:
# Check for issues that might need cleanup
print("=== Data Quality Check ===\n")

# 1. Check for duplicate IDs
duplicates = df_combined[df_combined['ID'].duplicated(keep=False)]
print(f"Duplicate IDs: {len(duplicates)}")
if len(duplicates) > 0:
    print(duplicates[['ID', 'tanggal', 'stasiun', 'kategori']].head())

# 2. Check for missing values in key columns
print("\nMissing values in critical columns:")
print(df_combined[['ID', 'tanggal', 'stasiun', 'kategori']].isnull().sum())

# 3. Check for invalid station IDs
print("\nUnique station IDs:")
print(sorted(df_combined['stasiun'].unique()))

# 4. Check kategori values
print("\nKategori distribution:")
print(df_combined['kategori'].value_counts())

# 5. Check for invalid dates
print(f"\nInvalid dates (NaT): {df_combined['tanggal'].isna().sum()}")

# 6. Check for negative or unrealistic pollutant values
print("\nNegative pollutant values:")
for col in numeric_columns:
    neg_count = (df_combined[col] < 0).sum()
    if neg_count > 0:
        print(f"  {col}: {neg_count}")

=== Data Quality Check ===

Duplicate IDs: 345
                  ID    tanggal stasiun kategori
421  2016-01-12_DKI3 2016-01-12    DKI3     BAIK
422  2016-01-12_DKI4 2016-01-12    DKI4   SEDANG
423  2016-01-12_DKI3 2016-01-12    DKI3   SEDANG
424  2016-01-12_DKI4 2016-01-12    DKI4   SEDANG
425  2016-01-12_DKI2 2016-01-12    DKI2   SEDANG

Missing values in critical columns:
ID          0
tanggal     0
stasiun     0
kategori    0
dtype: int64

Unique station IDs:
['DKI1', 'DKI2', 'DKI3', 'DKI4', 'DKI5']

Kategori distribution:
kategori
SEDANG                2938
BAIK                   911
TIDAK SEHAT            778
SANGAT TIDAK SEHAT      36
Name: count, dtype: int64

Invalid dates (NaT): 0

Negative pollutant values:


# Data Wilayah Administrasi DKI Jakarta

In [9]:
import sqlite3
import pandas as pd
import requests
import re

# Download the SQL file
url = "https://raw.githubusercontent.com/cahyadsn/wilayah/refs/heads/master/db/wilayah.sql"
response = requests.get(url)
response.raise_for_status()
sql_content = response.text

# Clean the SQL content
clean_sql = re.sub(r'^/\*.*?\*/\s*', '', sql_content, flags=re.DOTALL | re.MULTILINE).strip()

# Additional cleaning
clean_sql = re.sub(r'--.*?$', '', clean_sql, flags=re.MULTILINE)  # Remove -- comments
clean_sql = re.sub(r'ENGINE=\w+\s*', '', clean_sql, flags=re.IGNORECASE)  # Remove ENGINE
clean_sql = re.sub(r'DEFAULT CHARSET=\w+\s*', '', clean_sql, flags=re.IGNORECASE)  # Remove CHARSET
clean_sql = clean_sql.replace('`', '')  # Remove backticks

In [10]:
print(clean_sql)





DROP TABLE IF EXISTS wilayah;
CREATE TABLE IF NOT EXISTS wilayah (
    kode varchar(13) NOT NULL,
    nama varchar(100) NOT NULL,
    PRIMARY KEY (kode)
) ;
CREATE INDEX wilayah_name_idx ON wilayah (nama);






INSERT INTO wilayah (kode, nama)
VALUES
('11','Aceh'),
('11.01','Kabupaten Aceh Selatan'),
('11.01.01','Bakongan'),
('11.01.01.2001','Keude Bakongan'),
('11.01.01.2002','Ujong Mangki'),
('11.01.01.2003','Ujong Padang'),
('11.01.01.2004','Gampong Drien'),
('11.01.01.2015','Darul Ikhsan'),
('11.01.01.2016','Padang Beurahan'),
('11.01.01.2017','Gampong Baro'),
('11.01.02','Kluet Utara'),
('11.01.02.2001','Fajar Harapan'),
('11.01.02.2002','Krueng Batee'),
('11.01.02.2003','Pasi Kuala Asahan'),
('11.01.02.2004','Gunong Pulo'),
('11.01.02.2005','Pulo Ie I'),
('11.01.02.2006','Jambo Manyang'),
('11.01.02.2007','Simpang Empat'),
('11.01.02.2008','Limau Purut'),
('11.01.02.2009','Pulo Kambing'),
('11.01.02.2010','Kampung Paya'),
('11.01.02.2011','Krueng Batu'),
('11.01.02.2012','Kr

In [11]:
# Create SQLite connection
conn = sqlite3.connect(':memory:')  # Use in-memory database
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS wilayah (
    kode TEXT PRIMARY KEY,
    nama TEXT
)
""")
conn.commit()

# Split into statements and execute
statements = [s.strip() for s in clean_sql.split(';') if s.strip()]

executed = 0
errors = 0

for stmt in statements:
    if not stmt:
        continue
    
    upper = stmt.upper()
    
    # Skip non-essential statements
    if any(x in upper for x in ['CREATE INDEX', 'DROP TABLE', 'SET ', 'KEY ', 'LOCK ', 'UNLOCK']):
        continue
    
    # Only process CREATE TABLE and INSERT statements
    if upper.startswith('INSERT INTO'):
        try:
            cursor.execute(stmt)
            executed += 1
            if executed % 1000 == 0:
                print(f"Executed {executed} statements...")
        except Exception as e:
            errors += 1
            if errors < 5:  # Only print first few errors
                print(f"Error: {e}")
                print(f"Statement: {stmt[:100]}...")

conn.commit()
print(f"\nTotal executed: {executed}, Errors: {errors}")

# Load into DataFrame
df_wilayah = pd.read_sql_query("SELECT * FROM wilayah", conn)

print(f"\nLoaded {len(df_wilayah)} rows")
print("\nSample data:")
print(df_wilayah.head(10))

print("\nDKI Jakarta regions (code starts with 31):")
df_jakarta = df_wilayah[df_wilayah['kode'].str.startswith('31')]
print(df_jakarta.head(20))


Total executed: 38, Errors: 0

Loaded 91599 rows

Sample data:
            kode                    nama
0             11                    Aceh
1          11.01  Kabupaten Aceh Selatan
2       11.01.01                Bakongan
3  11.01.01.2001          Keude Bakongan
4  11.01.01.2002            Ujong Mangki
5  11.01.01.2003            Ujong Padang
6  11.01.01.2004           Gampong Drien
7  11.01.01.2015            Darul Ikhsan
8  11.01.01.2016         Padang Beurahan
9  11.01.01.2017            Gampong Baro

DKI Jakarta regions (code starts with 31):
                kode                                     nama
27687             31            Daerah Khusus Ibukota Jakarta
27688          31.01  Kabupaten Administrasi Kepulauan Seribu
27689       31.01.01                   Kepulauan Seribu Utara
27690  31.01.01.1001                           Pulau Panggang
27691  31.01.01.1002                             Pulau Kelapa
27692  31.01.01.1003                            Pulau Harapan
27693  

In [12]:
df_jakarta[~df_jakarta['kode'].str.match(r'^31\.01')]

,kode,nama
27687,31,Daerah Khusus Ibukota Jakarta
27697,31.71,Kota Administrasi Jakarta Pusat
27698,31.71.01,Gambir
27699,31.71.01.1001,Gambir
27700,31.71.01.1002,Cideng
...,...,...
28000,31.75.10.1004,Munjul
28001,31.75.10.1005,Setu
28002,31.75.10.1006,Bambu Apus
28003,31.75.10.1007,Lubang Buaya


In [13]:
df_jakarta[df_jakarta['kode'].str.match(r'^31\.\d{2}$')]

,kode,nama
27688,31.01,Kabupaten Administrasi Kepulauan Seribu
27697,31.71,Kota Administrasi Jakarta Pusat
27750,31.72,Kota Administrasi Jakarta Utara
27788,31.73,Kota Administrasi Jakarta Barat
27853,31.74,Kota Administrasi Jakarta Selatan
27929,31.75,Kota Administrasi Jakarta Timur


In [14]:
df_jakarta[df_jakarta['kode'].str.match(r'^31\.(?!01$)\d{2}$')]

,kode,nama
27697,31.71,Kota Administrasi Jakarta Pusat
27750,31.72,Kota Administrasi Jakarta Utara
27788,31.73,Kota Administrasi Jakarta Barat
27853,31.74,Kota Administrasi Jakarta Selatan
27929,31.75,Kota Administrasi Jakarta Timur


# Data Kualitas Air Sungai

In [3]:
sungai = pd.read_csv('data/kualitas-air-sungai/data-kualitas-air-sungai-komponen-data.csv')

In [16]:
sungai.columns

Index(['periode_data', 'periode_pemantauan', 'bulan_sampling', 'titik_sampel',
       'nama_sungai', 'alamat', 'latitude', 'longitude', 'jenis_parameter',
       'parameter', 'baku_mutu', 'hasil_pengukuran'],
      dtype='str')

In [4]:
params_sungai = ['Kimia', 'Fisika', 'Biologi'] 

index_columns = ['periode_data', 'periode_pemantauan', 'bulan_sampling', 'titik_sampel',
       'nama_sungai', 'alamat', 'latitude', 'longitude']
parameter_columns = 'parameter'
value_columns = 'hasil_pengukuran'

In [18]:
sungai_dfs = []
for param in params_sungai:
    _sungai_df = sungai[sungai['jenis_parameter'] == param].sort_values(by=['periode_data', 'periode_pemantauan', 'bulan_sampling', 'titik_sampel'])
    _sungai_df = _sungai_df.pivot(index=index_columns, columns=parameter_columns, values=value_columns)
    # _sungai_df.columns = [f'{val}_{par}' for val, par in _sungai_df.columns]
    _sungai_df = _sungai_df.reset_index()
    print(len(_sungai_df))
    sungai_dfs.append(_sungai_df)

480
480
480


In [19]:
from functools import reduce

common_cols = ['periode_data', 'periode_pemantauan', 'bulan_sampling', 'titik_sampel', 'nama_sungai', 'alamat', 'latitude', 'longitude']

sungai_df = reduce(lambda left, right: pd.merge(left, right, on=common_cols, how='inner'), sungai_dfs)

In [20]:
sungai_df.columns

Index(['periode_data', 'periode_pemantauan', 'bulan_sampling', 'titik_sampel',
       'nama_sungai', 'alamat', 'latitude', 'longitude', 'Amoniak', 'BOD',
       'COD', 'Cd', 'Cr6', 'Cu', 'DO', 'F', 'Fenol', 'H2S', 'Hg', 'Klorida',
       'Klorin Bebas', 'Klorin Bebas n In Situ', 'MBAS', 'Minyak dan Lemak',
       'Ni', 'Nitrat', 'Nitrit', 'Pb', 'Sianida', 'Sulfat', 'Total N',
       'Total P', 'Zn', 'pH', 'TDS', 'TSS', 'Warna', 'Fecal Coliform',
       'Total Coliform'],
      dtype='str', name='parameter')

In [21]:
sungai_df

parameter,periode_data,periode_pemantauan,bulan_sampling,titik_sampel,nama_sungai,alamat,latitude,longitude,Amoniak,BOD,...,Sulfat,Total N,Total P,Zn,pH,TDS,TSS,Warna,Fecal Coliform,Total Coliform
0,2024,Periode 1,2,ANK 1,Angke,Jl. Duri Kosambi Ciledug,-6.175882,106.728695,2.558,5.40,...,23.22,7.559,0.474,0.011,6.7648,208.0,70.0,7.0,16000.0,16000.0
1,2024,Periode 1,2,ANK 2,Angke,Jl. Daan Mogot Pesing,-6.158863,106.763746,3.330,7.00,...,17.46,9.030,0.624,0.027,6.7518,392.0,21.0,8.0,1700.0,1700.0
2,2024,Periode 1,2,ANK 3,Angke,Jl. Tubagus Angke,-6.157758,106.767695,3.082,6.90,...,11.31,8.778,0.595,0.012,6.7562,358.0,26.0,9.0,17000.0,17000.0
3,2024,Periode 1,2,BLC 1,Blencong,Jl. Rorotan Marunda,-6.118484,106.956440,4.786,7.00,...,25.74,9.021,1.190,0.019,8.4571,744.0,25.0,3.0,1600001.0,1600001.0
4,2024,Periode 1,2,BRN 1,Buaran,Jl. Kalimalang Depo Bangunan Pondok Kelapa,-6.246067,106.920545,4.894,5.90,...,13.54,6.509,0.839,0.008,6.0204,338.0,9.0,5.0,430000.0,1600001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2024,Periode 4,10,STR 7,Sunter,Jl. Yos Sodarso Kodamar,-6.155556,106.886139,18.950,32.51,...,25.90,27.480,1.990,0.009,7.4450,364.5,58.0,73.0,14000000.0,28000000.0
476,2024,Periode 4,10,STR 8,Sunter,Bogasari Koja,-6.118972,106.896250,15.100,44.99,...,56.10,18.210,1.450,0.009,7.4500,2056.0,45.0,68.0,2200000.0,3000000.0
477,2024,Periode 4,10,TRB 1,Tarum Barat,Jl. Kalimalang Batas DKI Jakarta Ruko Caman,-6.249899,106.952948,0.350,11.55,...,29.30,1.310,2.300,0.020,7.2150,151.4,27.0,5.0,3500.0,9000.0
478,2024,Periode 4,10,TRB 2,Tarum Barat,Jl. Kalimalang Universitas Borobudur,-6.242500,106.897076,0.280,10.74,...,24.40,1.220,2.300,0.010,7.2800,150.7,29.0,6.0,2200.0,5000.0


# Data Batas Wilayah Administrasi DKI Jakarta

In [29]:
import sqlite3
import pandas as pd
import requests

# ────────────────────────────────────────────────
# Step 1: Download the SQL file content
# ────────────────────────────────────────────────
url = "https://raw.githubusercontent.com/cahyadsn/wilayah_boundaries/refs/heads/main/db/kab/wilayah_boundaries_kab_31.sql"
response = requests.get(url)
response.raise_for_status()          # fail early if download error
sql_content = response.text

# Clean the SQL content
clean_sql = re.sub(r'^/\*.*?\*/\s*', '', sql_content, flags=re.DOTALL | re.MULTILINE).strip()

# Additional cleaning
clean_sql = re.sub(r'--.*?$', '', clean_sql, flags=re.MULTILINE)  # Remove -- comments
clean_sql = re.sub(r'ENGINE=\w+\s*', '', clean_sql, flags=re.IGNORECASE)  # Remove ENGINE
clean_sql = re.sub(r'DEFAULT CHARSET=\w+\s*', '', clean_sql, flags=re.IGNORECASE)  # Remove CHARSET
clean_sql = clean_sql.replace('`', '')  # Remove backticks

In [30]:
print(clean_sql)

DELETE FROM wilayah_boundaries WHERE LENGTH(kode)=5 AND LEFT(kode,2)='31';
INSERT INTO wilayah_boundaries(kode,nama,lat,lng,path) VALUES
	('31.01','Kabupaten Administrasi Kepulauan Seribu',-5.700366519193108,106.57895590245003,'[[[[-6.038562013426122,106.73588396767106],[-6.039344904743497,106.73646467160737],[-6.040184090922977,106.73542068731493],[-6.039027196747674,106.73535096917195],[-6.038562013426122,106.73588396767106]]],[[[-6.034403670592851,106.74658012187518],[-6.035278381791879,106.74798335294946],[-6.0373121302504975,106.74663238957318],[-6.0357235965667035,106.74556591143768],[-6.034403670592851,106.74658012187518]]],[[[-6.032174096557234,106.73528225287369],[-6.033060086154033,106.73664150530128],[-6.034900663834523,106.73526236976262],[-6.033580505835604,106.73336207440022],[-6.032174096557234,106.73528225287369]]],[[[-6.026508405438483,106.74536916225709],[-6.026544236227486,106.74483908385672],[-6.02498356983581,106.74512072723982],[-6.025320292197715,106.745499501900

In [1]:
# ────────────────────────────────────────────────
# Step 2: Parse and load boundary data into DataFrame
# ────────────────────────────────────────────────

# Extract INSERT statements and parse them
import re

# Find all INSERT statements
insert_pattern = r"INSERT INTO wilayah_boundaries_kab VALUES \('([^']+)','([^']+)','([^']+)','([^']+)','([^']+)'\)"
matches = re.findall(insert_pattern, clean_sql)

print(f"Found {len(matches)} boundary records")

# Create DataFrame directly from parsed data
df_batas_wilayah = pd.DataFrame(matches, columns=['kode', 'nama', 'lat', 'lng', 'geom'])

print(f"\nLoaded {len(df_batas_wilayah)} rows")
print("\nSample data:")
print(df_batas_wilayah.head())
print("\nColumns:", df_batas_wilayah.columns.tolist())

NameError: name 'clean_sql' is not defined

In [ ]:
# ────────────────────────────────────────────────
# Step 3: Convert to GeoDataFrame with proper geometries
# ────────────────────────────────────────────────
from shapely import wkt
import geopandas as gpd

# Parse WKT geometry strings
df_batas_wilayah['geometry'] = df_batas_wilayah['geom'].apply(wkt.loads)

# Create GeoDataFrame
gdf_boundaries = gpd.GeoDataFrame(df_batas_wilayah, geometry='geometry', crs='EPSG:4326')

print(f"Created GeoDataFrame with {len(gdf_boundaries)} boundaries")
print("\nKota/Kabupaten in DKI Jakarta:")
print(gdf_boundaries[['kode', 'nama']].to_string())

# Plot boundaries to visualize
ax = gdf_boundaries.plot(figsize=(10, 8), edgecolor='black', facecolor='lightblue', alpha=0.5)
ax.set_title('DKI Jakarta - Kota/Kabupaten Boundaries')
gdf_boundaries.apply(lambda x: ax.annotate(text=x['nama'], xy=x.geometry.centroid.coords[0], 
                                            ha='center', fontsize=8), axis=1)

In [ ]:
# ────────────────────────────────────────────────
# Step 4: Spatial join to assign kota/kabupaten to river samples
# ────────────────────────────────────────────────

# Convert sungai_df to GeoDataFrame
gdf_sungai = gpd.GeoDataFrame(
    sungai_df,
    geometry=gpd.points_from_xy(sungai_df.longitude, sungai_df.latitude),
    crs='EPSG:4326'
)

# Perform spatial join (find which boundary each point falls within)
gdf_sungai_with_region = gpd.sjoin(
    gdf_sungai, 
    gdf_boundaries[['kode', 'nama', 'geometry']], 
    how='left', 
    predicate='within'
)

# Rename columns for clarity
gdf_sungai_with_region = gdf_sungai_with_region.rename(columns={
    'kode': 'kode_wilayah',
    'nama': 'kota_kabupaten'
})

# Drop duplicate geometry columns if any
if 'index_right' in gdf_sungai_with_region.columns:
    gdf_sungai_with_region = gdf_sungai_with_region.drop(columns=['index_right'])

print(f"Assigned regions to {len(gdf_sungai_with_region)} river samples")
print("\nSamples by Kota/Kabupaten:")
print(gdf_sungai_with_region['kota_kabupaten'].value_counts())

# Show samples that couldn't be assigned (outside boundaries)
unassigned = gdf_sungai_with_region[gdf_sungai_with_region['kota_kabupaten'].isna()]
print(f"\nUnassigned samples: {len(unassigned)}")
if len(unassigned) > 0:
    print(unassigned[['titik_sampel', 'nama_sungai', 'latitude', 'longitude']].head())

# Convert back to regular DataFrame if needed
sungai_df_with_region = pd.DataFrame(gdf_sungai_with_region.drop(columns='geometry'))
sungai_df_with_region